<a href="https://colab.research.google.com/github/cliu-15/R250-Autoencoders-czl22/blob/master/AE_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# New Section

Read-in / Pre-process Dataset

In [0]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/My Drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision
import sklearn.decomposition

import IPython
from utils import interrupted, enumerate_cycle
from utils import mnist, mnist_batched
from intro_ae import SimpleAE

PCA with dataset

PCA-like autoencoder (based on paper)

Semi-linear nested dropout autoencoder (based on paper)

B-VAE (based on paper)

Other types of AE???

# Notes

Project 2. PCA produces an ordered list of orthogonal latent variables. Compare approaches for achieving the same with autoencoders. 
Relevant papers:
*   A PCA-like autoencoder (https://arxiv.org/abs/1904.01277)
*   Learning Ordered Representations with Nested Dropout (https://arxiv.org/abs/1402.0915)
*   The β-VAE paper we studied





Papers notes:


PCA background:
- Ordered list of othogonal latent variables